In [1]:
import pandas as pd
from collections import Counter
import seaborn as sns
import numpy 
import scipy

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer,f1_score
from sklearn.linear_model import LogisticRegression
from sys import getsizeof

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

from sklearn.model_selection import train_test_split
from rfpimp import *

scorer = make_scorer(f1_score, needs_proba=False,
                     greater_is_better=True,average='micro')

In [37]:
df = pd.read_csv('../input/train_values.csv')
train_target = pd.read_csv('../input/train_labels.csv')

print(Counter(train_target.damage_grade))

y = train_target.damage_grade

Counter({2: 148259, 3: 87218, 1: 25124})


In [10]:
comb =  pd.merge(df, train_target, on='building_id')

In [15]:
comb.shape

(260601, 40)

In [12]:
id_cols = [i for i in comb.columns if 'id' in i]

In [13]:
id_cols

['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']

In [14]:
comb.drop(columns=id_cols).drop_duplicates().shape

(164527, 36)

In [19]:
id_cols_tar = id_cols+['damage_grade']

In [20]:
comb.drop(columns= id_cols_tar).drop_duplicates().shape

(145516, 35)

In [17]:
164527 - 145516

19011

In [21]:
other_cols = [i for i in comb.columns if i not in id_cols_tar]

In [26]:
comb = comb.sort_values(by=other_cols)

In [30]:
comb['dup'] = comb.duplicated(subset=other_cols,keep=False)

In [31]:
comb = comb[comb.dup]

In [32]:
comb.shape

(141615, 41)

In [33]:
comb.iloc[:1000].to_csv('../output/dupl_check.csv', index=False
                    )

In [46]:
df.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0


In [47]:
min(df.building_id)

4

In [48]:
max(df.building_id)

1052934

In [49]:
df.shape

(260601, 39)

In [50]:
test = pd.read_csv("../input/test_values.csv")

In [51]:
min(test.building_id), max(test.building_id), test.shape

(7, 1052923, (86868, 39))

In [52]:
train_target.sort_values('building_id', inplace=True)

In [65]:
test.shape, test.drop(columns=id_cols).drop_duplicates().shape

((86868, 39), (58104, 35))

In [66]:
df.drop(columns=['building_id']).drop_duplicates().shape

(260601, 38)

In [67]:
df.drop(columns=['building_id']).drop_duplicates().shape

(244410, 38)

In [70]:
df1 = df.drop(columns=['building_id'])

In [71]:
df2 = test.drop(columns=['building_id'])

In [72]:
df1.shape, df2.shape

((260601, 38), (86868, 38))

In [81]:
df_all = pd.concat([df1, df2], axis=0)

In [82]:
df_all.shape

(347469, 38)

In [83]:
df_all.drop_duplicates().shape

(321056, 38)

In [84]:
347469 - 321056

26413

In [85]:
df1.shape[0] - df1.drop_duplicates().shape[0]

16191

In [86]:
df1.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [90]:
df[id_cols[1:]].shape

(260601, 3)

In [91]:
df[id_cols[1:]].drop_duplicates().shape

(11595, 3)

In [103]:
id_cols[1:]

['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']

In [112]:
comb.groupby(by=id_cols[1:], as_index=False).agg({'damage_grade':['count','mean']}).to_csv('../output/combs.csv')

In [114]:
df1 = df[id_cols[1:]]
df2 = test[id_cols[1:]]

In [116]:
pd.concat([df1, df2]).drop_duplicates().shape

(11861, 3)